# Importamos librerias

In [1]:
import pandas as pd
import os
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Guardar resultados en la carpeta output

In [2]:
output_dir = '../output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Carpeta '{output_dir}' creada")
else:
    print(f"Usando carpeta existente: {output_dir}")

print("=== CONFIGURACIÓN COMPLETADA ===")
print(f"Directorio de trabajo: {os.getcwd()}")
print(f"Carpeta de salida: {output_dir}")

Usando carpeta existente: ../output
=== CONFIGURACIÓN COMPLETADA ===
Directorio de trabajo: c:\Users\Usuario\Desktop\CICLOWAA8\data science\web_scrapping\code
Carpeta de salida: ../output


# Parte 1: Web Scraping Top Stock Gainers from Yahoo Finance

In [3]:
def scrape_yahoo_gainers():
    """Función optimizada para scrapear los top 50 gainers de Yahoo Finance"""
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Chrome(options=options)
    symbols, names = [], []
    
    try:
        driver.get("https://finance.yahoo.com/markets/stocks/gainers")
        driver.maximize_window()
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table")))
        time.sleep(5)
        
        page = 1
        while len(symbols) < 50:
            print(f"Procesando página {page}...")
            rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
            
            for row in rows:
                if len(symbols) >= 50:
                    break
                try:
                    cells = row.find_elements(By.TAG_NAME, "td")
                    if len(cells) >= 2:
                        # Obtener symbol y name
                        symbol_cell = cells[0]
                        symbol = symbol_cell.find_element(By.TAG_NAME, "a").text.strip() if symbol_cell.find_elements(By.TAG_NAME, "a") else symbol_cell.text.strip()
                        name = cells[1].text.strip()
                        
                        # Validar y agregar
                        symbol = symbol.replace('$', '').replace(',', '').strip()
                        if symbol and name and symbol not in symbols and len(symbol) <= 6:
                            symbols.append(symbol)
                            names.append(name)
                            print(f"{len(symbols)}. {symbol} - {name}")
                except:
                    continue
            
            # Buscar siguiente página si es necesario
            if len(symbols) < 50:
                next_found = False
                for selector in ["button[aria-label*='next']", "a[aria-label*='next']"]:
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, selector)
                        if next_button.is_enabled():
                            driver.execute_script("arguments[0].click();", next_button)
                            next_found = True
                            break
                    except:
                        continue
                
                if not next_found:
                    break
                page += 1
                time.sleep(4)
    
    except Exception as e:
        print(f"Error durante el scraping: {e}")
    finally:
        driver.quit()
    
    return symbols[:50], names[:50]

# EJECUCIÓN PARTE 1
if __name__ == "__main__":
    print("=== PARTE 1: WEB SCRAPING ===")
    symbols_list, names_list = scrape_yahoo_gainers()
    
    # Crear DataFrame con los resultados
    gainers_df = pd.DataFrame({'Symbol': symbols_list, 'Name': names_list})
    print(f"\nTotal acciones obtenidas: {len(gainers_df)}")
    
    # Guardar resultados con timestamp único en carpeta output
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    gainers_df.to_csv(os.path.join(output_dir, f'top_50_gainers_{timestamp}.csv'), index=False)
    gainers_df.to_csv(os.path.join(output_dir, 'latest_gainers.csv'), index=False)
    
    print("✅ PARTE 1 COMPLETADA - Archivos guardados en 'output/':")
    print(f"  - top_50_gainers_{timestamp}.csv")
    print("  - latest_gainers.csv")

=== PARTE 1: WEB SCRAPING ===
Procesando página 1...
1. CYTK - Cytokinetics, Incorporated
2. UTHR - United Therapeutics Corporation
3. IONS - Ionis Pharmaceuticals, Inc.
4. ARWR - Arrowhead Pharmaceuticals, Inc.
5. SBSW - Sibanye Stillwater Limited
6. HMY - Harmony Gold Mining Company Limited
7. JOYY - JOYY Inc.
8. IREN - IREN Limited
9. MENS - Jyong Biotech Ltd.
10. AL - Air Lease Corporation
11. ULTA - Ulta Beauty, Inc.
12. ONC - BeOne Medicines AG
13. INSM - Insmed Incorporated
14. DOOO - BRP Inc.
15. QXO - QXO, Inc.
16. RARE - Ultragenyx Pharmaceutical Inc.
17. EQX - Equinox Gold Corp.
18. PTCT - PTC Therapeutics, Inc.
19. BEKE - KE Holdings Inc.
20. CRNX - Crinetics Pharmaceuticals, Inc.
21. CROX - Crocs, Inc.
22. AGI - Alamos Gold Inc.
23. CIFR - Cipher Mining Inc.
24. LI - Li Auto Inc.
25. BHC - Bausch Health Companies Inc.
Procesando página 2...
26. BIIB - Biogen Inc.
27. ASND - Ascendis Pharma A/S
28. MUR - Murphy Oil Corporation
29. AU - AngloGold Ashanti plc
30. GMAB - Genma

# Parte 2: Historical Data Retrieval

In [4]:
def retrieve_historical_data(symbols_list, period="1y", interval="1mo"):
    """Función optimizada para obtener datos históricos"""
    print(f"Descargando datos históricos para {len(symbols_list)} símbolos...")
    
    historical_data = {}
    successful_count = 0
    
    for i, symbol in enumerate(symbols_list, 1):
        try:
            hist_data = yf.Ticker(symbol).history(period=period, interval=interval)
            if not hist_data.empty and len(hist_data['Close']) > 0:
                historical_data[symbol] = hist_data['Close']
                successful_count += 1
                print(f"[{i}/{len(symbols_list)}] {symbol} ✅")
            else:
                print(f"[{i}/{len(symbols_list)}] {symbol} ❌ Sin datos")
        except Exception as e:
            print(f"[{i}/{len(symbols_list)}] {symbol} ❌ Error: {str(e)[:30]}...")
        time.sleep(0.1)  # Evitar sobrecarga de la API
    
    if historical_data:
        # Crear DataFrame combinado
        combined_df = pd.DataFrame(historical_data)
        combined_df = combined_df.dropna(how='all')
        
        # Limpiar fechas futuras y normalizar
        today = pd.Timestamp.now().tz_localize(None)
        combined_df.index = pd.to_datetime(combined_df.index).tz_localize(None)
        combined_df = combined_df.loc[combined_df.index <= today].sort_index(ascending=False)
        
        print(f"\nDataFrame final: {combined_df.shape[0]} fechas x {combined_df.shape[1]} símbolos")
        print(f"Símbolos exitosos: {successful_count}/{len(symbols_list)}")
        return combined_df
    else:
        print("❌ No se obtuvieron datos para ningún símbolo")
        return pd.DataFrame()

# EJECUCIÓN PARTE 2
if __name__ == "__main__":
    print("\n=== PARTE 2: DATOS HISTÓRICOS ===")
    
    # Cargar símbolos de la Parte 1 desde carpeta output
    try:
        gainers_df = pd.read_csv(os.path.join(output_dir, 'latest_gainers.csv'))
        symbols_list = gainers_df['Symbol'].tolist()
        print(f"Símbolos cargados desde 'output/': {len(symbols_list)}")
    except FileNotFoundError:
        print("❌ Ejecuta primero la Parte 1")
        symbols_list = []
    
    if symbols_list:
        # Descargar datos históricos
        historical_df = retrieve_historical_data(symbols_list)
        
        if not historical_df.empty:
            # Guardar datos históricos en carpeta output
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            historical_df.to_csv(os.path.join(output_dir, f'monthly_historical_data_{timestamp}.csv'))
            historical_df.to_csv(os.path.join(output_dir, 'monthly_historical_data.csv'))
            print(f"✅ Datos guardados: {historical_df.shape[0]} fechas x {historical_df.shape[1]} símbolos")
            
            print("✅ PARTE 2 COMPLETADA - Archivos guardados en 'output/':")
            print(f"  - monthly_historical_data_{timestamp}.csv")
            print("  - monthly_historical_data.csv")
        else:
            print("❌ PARTE 2 FALLÓ - No se obtuvieron datos históricos")


=== PARTE 2: DATOS HISTÓRICOS ===
Símbolos cargados desde 'output/': 41
Descargando datos históricos para 41 símbolos...
[1/41] CYTK ✅
[2/41] UTHR ✅
[3/41] IONS ✅
[4/41] ARWR ✅
[5/41] SBSW ✅
[6/41] HMY ✅
[7/41] JOYY ✅
[8/41] IREN ✅
[9/41] MENS ✅
[10/41] AL ✅
[11/41] ULTA ✅
[12/41] ONC ✅
[13/41] INSM ✅
[14/41] DOOO ✅
[15/41] QXO ✅
[16/41] RARE ✅
[17/41] EQX ✅
[18/41] PTCT ✅
[19/41] BEKE ✅
[20/41] CRNX ✅
[21/41] CROX ✅
[22/41] AGI ✅
[23/41] CIFR ✅
[24/41] LI ✅
[25/41] BHC ✅
[26/41] BIIB ✅
[27/41] ASND ✅
[28/41] MUR ✅
[29/41] AU ✅
[30/41] GMAB ✅
[31/41] RL ✅
[32/41] W ✅
[33/41] PFGC ✅
[34/41] HL ✅
[35/41] ETOR ✅
[36/41] APGE ✅
[37/41] AG ✅
[38/41] GFI ✅
[39/41] AEO ✅
[40/41] LQDA ✅
[41/41] ARX ✅

DataFrame final: 12 fechas x 41 símbolos
Símbolos exitosos: 41/41
✅ Datos guardados: 12 fechas x 41 símbolos
✅ PARTE 2 COMPLETADA - Archivos guardados en 'output/':
  - monthly_historical_data_20250902_114405.csv
  - monthly_historical_data.csv


# Parte 3: Portfolio Construction & Analysis

In [5]:
def analyze_momentum_portfolio():
    """Función optimizada para construir y analizar portafolio de momentum"""
    print("=== ANÁLISIS DE PORTAFOLIO DE MOMENTUM ===")
    
    # Cargar datos históricos desde carpeta output
    try:
        historical_df = pd.read_csv(os.path.join(output_dir, 'monthly_historical_data.csv'), index_col=0, parse_dates=True)
        historical_df = historical_df.sort_index(ascending=True)
        print(f"Datos cargados desde 'output/': {historical_df.shape[0]} fechas x {historical_df.shape[1]} símbolos")
    except FileNotFoundError:
        print("❌ No se encontró 'output/monthly_historical_data.csv'")
        print("Ejecuta primero las Partes 1 y 2")
        return None
    
    # Validar que tenemos suficientes datos
    if len(historical_df) < 6:
        print(f"❌ Datos insuficientes: solo {len(historical_df)} meses disponibles")
        return None
    
    # PASO 1: Dividir datos en períodos de 6 meses
    first_6_months = historical_df.iloc[:6]
    last_6_months = historical_df.iloc[6:12] if len(historical_df) >= 12 else historical_df.iloc[6:]
    
    print(f"Período selección: {first_6_months.index[0].strftime('%Y-%m-%d')} a {first_6_months.index[-1].strftime('%Y-%m-%d')}")
    print(f"Período análisis: {last_6_months.index[0].strftime('%Y-%m-%d')} a {last_6_months.index[-1].strftime('%Y-%m-%d')}")
    
    # PASO 2: Calcular rendimientos acumulados primeros 6 meses
    complete_stocks_first = first_6_months.dropna(axis=1)
    if len(complete_stocks_first.columns) < 10:
        print(f"⚠️ Solo {len(complete_stocks_first.columns)} acciones con datos completos")
    
    cumulative_returns = ((complete_stocks_first.iloc[-1] / complete_stocks_first.iloc[0]) - 1) * 100
    cumulative_returns = cumulative_returns.sort_values(ascending=False)
    
    # PASO 3: Seleccionar top 10 para el portafolio
    num_stocks = min(10, len(cumulative_returns))
    portfolio_symbols = cumulative_returns.head(num_stocks).index.tolist()
    
    print(f"\n✅ Portafolio seleccionado (Top {num_stocks}):")
    for i, symbol in enumerate(portfolio_symbols, 1):
        print(f"  {i:2d}. {symbol:6s} - {cumulative_returns[symbol]:+7.2f}%")
    
    # PASO 4: Analizar performance en período de análisis
    portfolio_data = last_6_months[portfolio_symbols].dropna(axis=1)
    if portfolio_data.empty:
        print("❌ No hay datos suficientes para análisis de performance")
        return None
    
    # Calcular rendimientos mensuales individuales y del portafolio
    individual_monthly_returns = portfolio_data.pct_change().fillna(0) * 100
    portfolio_monthly_returns = individual_monthly_returns.mean(axis=1)  # Equal-weighted
    
    # PASO 5: Calcular estadísticas del portafolio
    total_return = ((1 + portfolio_monthly_returns/100).cumprod().iloc[-1] - 1) * 100
    volatility = portfolio_monthly_returns.std() * np.sqrt(12)  # Anualizada
    avg_monthly_return = portfolio_monthly_returns.mean()
    
    print(f"\n📊 ESTADÍSTICAS DEL PORTAFOLIO:")
    print(f"  • Rendimiento total:       {total_return:+7.2f}%")
    print(f"  • Rendimiento mensual:     {avg_monthly_return:+7.2f}%")
    print(f"  • Volatilidad anualizada:  {volatility:7.2f}%")
    
    # PASO 6: Mostrar rendimientos individuales período análisis
    individual_cumulative = ((portfolio_data.iloc[-1] / portfolio_data.iloc[0]) - 1) * 100
    print(f"\n📈 RENDIMIENTOS INDIVIDUALES (Período análisis):")
    for symbol in individual_cumulative.sort_values(ascending=False).index:
        print(f"  {symbol:6s}: {individual_cumulative[symbol]:+7.2f}%")
    
    return {
        'portfolio_symbols': portfolio_data.columns.tolist(),
        'individual_monthly_returns': individual_monthly_returns,
        'portfolio_monthly_returns': portfolio_monthly_returns,
        'portfolio_cumulative_return': total_return,
        'portfolio_volatility': volatility
    }

# EJECUCIÓN PARTE 3
if __name__ == "__main__":
    print("\n=== PARTE 3: ANÁLISIS DE PORTAFOLIO ===")
    
    # Analizar portafolio de momentum
    results = analyze_momentum_portfolio()
    
    if results:
        print("\n✅ PARTE 3 COMPLETADA")
        print("\n🎯 ESTRATEGIA IMPLEMENTADA:")
        print("  - Selección: Top 10 con mayor momentum (primeros 6 meses)")
        print("  - Portafolio: Equal-weighted (10% por acción)")
        print("  - Análisis: Performance últimos 6 meses")
    else:
        print("\n❌ PARTE 3 FALLÓ - Revisa los datos de entrada")


=== PARTE 3: ANÁLISIS DE PORTAFOLIO ===
=== ANÁLISIS DE PORTAFOLIO DE MOMENTUM ===
Datos cargados desde 'output/': 12 fechas x 41 símbolos
Período selección: 2024-10-01 a 2025-03-01
Período análisis: 2025-04-01 a 2025-09-01

✅ Portafolio seleccionado (Top 10):
   1. HMY    -  +36.97%
   2. LQDA   -  +35.94%
   3. ONC    -  +34.31%
   4. GFI    -  +34.04%
   5. AU     -  +33.53%
   6. AGI    -  +32.68%
   7. PTCT   -  +27.66%
   8. ASND   -  +26.90%
   9. EQX    -  +24.19%
  10. JOYY   -  +23.29%

📊 ESTADÍSTICAS DEL PORTAFOLIO:
  • Rendimiento total:        +33.69%
  • Rendimiento mensual:       +5.18%
  • Volatilidad anualizada:    26.72%

📈 RENDIMIENTOS INDIVIDUALES (Período análisis):
  LQDA  : +104.15%
  GFI   :  +53.13%
  JOYY  :  +47.73%
  AU    :  +41.43%
  EQX   :  +37.01%
  ONC   :  +25.60%
  ASND  :  +18.33%
  AGI   :  +11.26%
  PTCT  :   +3.73%
  HMY   :   -9.89%

✅ PARTE 3 COMPLETADA

🎯 ESTRATEGIA IMPLEMENTADA:
  - Selección: Top 10 con mayor momentum (primeros 6 meses)
  -